##Installating Dependencies

In [ ]:
!pip install google_play_scraper

In [ ]:
!pip install tzlocal

In [ ]:
!pip install app_store_scraper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into a

##Loading Libraries

In [ ]:
import sys, os
import json
import csv
import pandas as pd
import numpy as np

In [ ]:
# for scraping app info and reviews from Google Play
from google_play_scraper import app, Sort, reviews

# for pretty printing data structures
from pprint import pprint

# for keeping track of timing
import datetime as dt
from tzlocal import get_localzone

# for building in wait times
import random
import time

# for scraping app info and reviews from App Store
from app_store_scraper import AppStore

##Initialize Variables for the app

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Make a new "Reviews" folder inside the "Outcome" folders

### Playstore Extractor

In [ ]:
def extract_playstore_reviews(app_name, app_id):
    # Get start time
    start = dt.datetime.now(tz=get_localzone())
    fmt= "%m/%d/%y - %T %p"

    print('---'*20)
    print(f'***** {app_name} started at {start.strftime(fmt)}')

    app_reviews = []
    count = 100
    batch_num = 0

    rvws, token = reviews(
        app_id,
        lang='en',
        country='us',
        sort=Sort.NEWEST,
        count=count
    )

    app_reviews.extend(rvws)
    batch_num += 1
    print(f'Batch {batch_num} completed.')

    time.sleep(random.randint(1, 5))

    pre_review_ids = [rvw['reviewId'] for rvw in app_reviews]

    for _ in range(4999):
        rvws, token = reviews(
            app_id,
            lang='en',
            country='us',
            sort=Sort.NEWEST,
            count=count,
            continuation_token=token
        )

        if not rvws:
            break

        new_review_ids = [rvw['reviewId'] for rvw in rvws]
        unique_ids = set(pre_review_ids) != set(pre_review_ids + new_review_ids)

        if not unique_ids:
            print(f'No new reviews. Completed {batch_num} batches.\n')
            break

        app_reviews.extend(rvws)
        batch_num += 1
        print(f'Batch {batch_num} completed.')

        pre_review_ids += new_review_ids

    for r in app_reviews:
        r['app_name'] = app_name
        r['app_id'] = app_id

    return app_reviews

In [ ]:
def save_reviews_to_csv(reviews, base_path='/content/drive/MyDrive/BFSI', filename='reviews.csv'):
    print(f"Attempting to save {len(reviews)} reviews to CSV.")
    if not reviews:
        print("No reviews to save. Exiting the function.")
        return

    # Create the directory if it does not exist
    os.makedirs(base_path, exist_ok=True)
    full_path = os.path.join(base_path, filename)
    print(f"Saving to {full_path}")

    keys = reviews[0].keys()
    with open(full_path, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(reviews)
    print("Save completed.")

In [ ]:
app_ids = [
    ('HSBC Bank', 'uk.co.hsbc.hsbcukmobilebanking'),
    ('IDFC First Bank', 'com.idfcfirstbank.optimus'),
    ('Axis Mobile', 'com.axis.mobile'),
    ('HDFC Bank', 'com.snapwork.hdfc')
]

all_reviews = []
for app_name, app_id in app_ids:
    app_reviews = extract_playstore_reviews(app_name, app_id)
    all_reviews.extend(app_reviews)

print(f"Total reviews extracted: {len(all_reviews)}")
save_reviews_to_csv(all_reviews)

------------------------------------------------------------
***** HSBC Bank started at 03/30/24 - 18:57:54 PM
Batch 1 completed.
------------------------------------------------------------
***** IDFC First Bank started at 03/30/24 - 18:57:57 PM
Batch 1 completed.
Batch 2 completed.
------------------------------------------------------------
***** Axis Mobile started at 03/30/24 - 18:58:00 PM
Batch 1 completed.
Batch 2 completed.
------------------------------------------------------------
***** HDFC Bank started at 03/30/24 - 18:58:03 PM
Batch 1 completed.
Batch 2 completed.
Batch 3 completed.
Batch 4 completed.
Total reviews extracted: 900
Attempting to save 900 reviews to CSV.
Saving to /content/drive/MyDrive/BFSI/reviews.csv
Save completed.


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/BFSI/reviews.csv')

In [ ]:
final_review_df = df[['content', 'score', 'thumbsUpCount']]

In [ ]:
!pip install nltk

In [ ]:
import pandas as pd
import nltk
nltk.download('punkt')

def preprocess_reviews(reviews):
    # List to collect each sentence as a separate DataFrame
    sentence_frames = []

    for index, row in reviews.iterrows():
        # Split the review content into sentences
        sentences = nltk.sent_tokenize(row['content'])

        for sentence in sentences:
            # Create a DataFrame for each sentence
            sentence_frame = pd.DataFrame({
                'content': [sentence],
                'score': [row['score']],
                'thumbsUpCount': [row['thumbsUpCount']]
            })
            sentence_frames.append(sentence_frame)

    # Concatenate all sentence DataFrames into a single DataFrame
    preprocessed_reviews = pd.concat(sentence_frames, ignore_index=True)

    return preprocessed_reviews

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
preprocessed_df = preprocess_reviews(df)

In [ ]:
preprocessed_df = pd.read_csv('/content/drive/MyDrive/BFSI/Financial.csv')

In [ ]:
preprocessed_df

,content,score,thumbsUpCount
0,1] Customer service is really nice.,3,0
1,2] It is not very easy to set up mobile bankin...,3,0
2,I understand it's for safety but still a hassle.,3,0
3,3] App itself is easy to use.,3,0
4,"4] Unfortunately, you can't view your card num...",3,0
...,...,...,...
1278,Worst experience logging issue every time,1,0
1279,It was good earlier but after update it forces...,1,0
1280,Perfect!,5,0
1281,!,5,0


In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 751.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_ckpt = "papluca/xlm-roberta-base-language-detection"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSequenceClassification.from_pretrained(model_ckpt)

In [ ]:
def detect_language(texts, tokenizer, model, batch_size=32):
    model.eval()
    languages = []
    id2lang = model.config.id2label

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)

        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Map prediction IDs to language codes
        batch_languages = [id2lang[pred.item()] for pred in predictions]
        languages.extend(batch_languages)

    return languages

In [ ]:
preprocessed_df['language'] = detect_language(preprocessed_df['content'].tolist(), tokenizer, model)

In [ ]:
preprocessed_df

,content,score,thumbsUpCount,language
0,1] Customer service is really nice.,3,0,en
1,2] It is not very easy to set up mobile bankin...,3,0,en
2,I understand it's for safety but still a hassle.,3,0,en
3,3] App itself is easy to use.,3,0,en
4,"4] Unfortunately, you can't view your card num...",3,0,en
...,...,...,...,...
1278,Worst experience logging issue every time,1,0,en
1279,It was good earlier but after update it forces...,1,0,en
1280,Perfect!,5,0,en
1281,!,5,0,hi


In [ ]:
preprocessed_df = preprocessed_df[preprocessed_df['language'] == 'en']

In [ ]:
preprocessed_df.to_csv('/content/drive/MyDrive/BFSI/final_reviews.csv', index=False)